<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/GSE108571_GSM2905291_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE108571_GSM2905291


This notebook uses the filtered count matrix resulting from the following google colab notebook. https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE108571_GSM2905291.ipynb Please run the notebook above and upload the results file in /content

In [1]:
# define the values for the analysis

# accession id for the data
id = "GSE108571"
samp_id = "GSM2905291"

# If only bam available files, set bam = True, Fill link and filename
bam = False

if bam:
  # Assign link to python variable
  link_to_bam = "ftp://ftp.sra.ebi.ac.uk/vol1/run/SRR638/SRR6389864/possorted_genome_bam.bam"
  
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename="possorted_genome_bam.bam"

if not bam:
  fastqs = ["ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR642/002/SRR6425162/SRR6425162_1.fastq.gz", \
            "ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR642/002/SRR6425162/SRR6425162_2.fastq.gz"]


In [2]:
no_samples = 1

sample_id = [samp_id] * no_samples

database_id = [id] * no_samples

tissue = ["brain organoid"] * no_samples

cell_type = ["developmental stage: day 32, stem cell line: H9"] * no_samples

condition = ["protocol: BMPi"] * no_samples

species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Muus et al 2020"] * no_samples

figure = ["Fig 1 a,b  ED Fig 1 a,b,c,d  ED Fig 2 a,b,c,d,e"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]

# Imports and installs

In [3]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 59.1MB 68kB/s 
     |████████████████████████████████| 122kB 29.9MB/s 
     |████████████████████████████████| 10.3MB 45.4MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 13.2MB 140kB/s 
     |████████████████████████████████| 112kB 43.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 


# Downloads: (bam (if bam) and index

In [4]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  !wget -- continue ${BAM_LINK}




In [5]:
if bam:
  # Convert to fastq
  !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $bam_filename ./fastqs\

  # Remove original bam file to save space
  !rm $bam_filename


In [6]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


In [7]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [8]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

/content
[2021-03-09 20:41:03,466]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
100% 2.23G/2.23G [01:39<00:00, 24.0MB/s]
[2021-03-09 20:42:43,874]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [9]:
print(fastqs)

['ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR642/002/SRR6425162/SRR6425162_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR642/002/SRR6425162/SRR6425162_2.fastq.gz']


# Process fastq files (modify kb command according to fastqs list)


In [12]:
# if no_samples == 1:

#   # Write the kb count command as a string with all fastqs of the list as an input
#   cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
#   + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
#   # Execute it
#   !$cmd

# # If more than one sample, iterate through fastqs accordingly
# else:

#   # Initializa counter for fastq files
#   j = 0

#   # Loop over samples for analysis
#   for i in range(no_samples):

#     fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
#     # Write the kb count command as a string
#     cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
#     --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

#     # Execute it
#     !$cmd

#     # Update j to move to the next set of fastq
#     j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


  adding: bus_raw_GSM2905291_from_gc/ (stored 0%)
  adding: bus_raw_GSM2905291_from_gc/run_info.json (deflated 42%)
  adding: bus_raw_GSM2905291_from_gc/kb_info.json (deflated 75%)
  adding: bus_raw_GSM2905291_from_gc/inspect.json (deflated 57%)
  adding: bus_raw_GSM2905291_from_gc/tcc_unfiltered/ (stored 0%)
  adding: bus_raw_GSM2905291_from_gc/tcc_unfiltered/adata.h5ad.gz (deflated 5%)
  adding: bus_raw_GSM2905291_from_gc/tcc_unfiltered/cells_x_tcc.mtx.gz (deflated 0%)
  adding: bus_raw_GSM2905291_from_gc/tcc_unfiltered/cells_x_tcc.barcodes.txt.gz (deflated 1%)
  adding: bus_raw_GSM2905291_from_gc/tcc_unfiltered/cells_x_tcc.ec.txt.gz (deflated 1%)
  adding: bus_raw_GSM2905291_from_gc/GSM2905291.unfiltered.bus.gz (deflated 4%)


In [ ]:
import time
time.sleep(650000)